Desarrollo Algoritmo para Prueba 1

1) Selección de Países y Recolección de Datos
Esta celda procesa los datos del recurso solar y condiciones meteorológicas de Chile, Sudáfrica e Indonesia.

In [6]:
#Importar Librerías
import polars as pl
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os

In [7]:
# Definir los pares de archivos por ubicación
ubicaciones = {
    'Sudafrica': {
        'lat': -29.19,
        'lon': 21.30,
        'elev': 0,  # Elevación por defecto
        'tz': 2,    # Zona horaria por defecto
        'archivos': ['1155731_-29.19_21.30_2018.csv', '1155731_-29.19_21.30_2019.csv']
    },
    'China': {
        'lat': 44.73,
        'lon': 87.66,
        'elev': 0,
        'tz': 8,
        'archivos': ['3480150_44.73_87.66_2018.csv', '3480150_44.73_87.66_2018.csv']
    },
    'Chile': {
        'lat': -23.84,
        'lon': -69.89,
        'elev': 735,  # Elevación real de Chile
        'tz': -4,
        'archivos': ['5815755_-23.84_-69.89_2018.csv', '5815755_-23.84_-69.89_2019.csv']
    }
}

# Procesar cada ubicación
for id_ubicacion, info in ubicaciones.items():
    print(f"\nProcesando ubicación {id_ubicacion} ({info['lat']}, {info['lon']})")
    
    # Lista para almacenar los DataFrames de cada año
    dfs = []
    
    # Procesar cada archivo de la ubicación
    for archivo in info['archivos']:
        # Leer el archivo CSV, saltando las primeras 2 filas que contienen metadatos
        df = pl.read_csv(archivo, skip_rows=2)
        
        # Seleccionar solo las columnas necesarias para PySAM
        df = df.select([
            pl.col('Year'),
            pl.col('Month'),
            pl.col('Day'),
            pl.col('Hour'),
            pl.col('Minute'),
            pl.col('DNI'),
            pl.col('GHI'),
            pl.col('DHI')
        ])
        
        # Agregar el DataFrame a la lista
        dfs.append(df)
    
    # Combinar los DataFrames de los dos años
    df_final = pl.concat(dfs)
    
    # Ordenar por fecha y hora
    df_final = df_final.sort(['Year', 'Month', 'Day', 'Hour', 'Minute'])
    
    # Crear el archivo de salida con el formato PySAM
    nombre_archivo = f'datos_{id_ubicacion}.csv'
    
    # Escribir los metadatos y los datos en un solo paso
    with open(nombre_archivo, 'w') as f:
        # Escribir metadatos en formato PySAM
        f.write(f"# Source: Solar Data for {id_ubicacion}\n")
        f.write(f"# Location: {id_ubicacion}\n")
        f.write(f"# Lat: {info['lat']}\n")
        f.write(f"# Lon: {info['lon']}\n")
        f.write(f"# Elev: {info['elev']}\n")
        f.write(f"# Time Zone: {info['tz']}\n")
        f.write(f"# Local Time Zone: {info['tz']}\n")
        f.write(f"# Data Format: TMY3\n\n")
        
        # Escribir encabezados y datos
        df_final.write_csv(f)
    
    print(f"\nLos datos han sido guardados en '{nombre_archivo}' en formato PySAM")

print("\nProcesamiento completado para todas las ubicaciones") 


Procesando ubicación Sudafrica (-29.19, 21.3)

Los datos han sido guardados en 'datos_Sudafrica.csv' en formato PySAM

Procesando ubicación China (44.73, 87.66)

Los datos han sido guardados en 'datos_China.csv' en formato PySAM

Procesando ubicación Chile (-23.84, -69.89)

Los datos han sido guardados en 'datos_Chile.csv' en formato PySAM

Procesamiento completado para todas las ubicaciones


Analisis de Recurso Solar

In [8]:
def cargar_datos(archivo):
    # Primero leemos los metadatos manualmente
    with open(archivo, 'r') as f:
        # Saltamos las líneas de metadatos
        for _ in range(9):
            next(f)
        
        # Leer el resto del archivo con polars
        df = pl.read_csv(
            archivo,
            skip_rows=9,  # Saltamos las líneas de metadatos
            dtypes={
                'Year': pl.Int64,
                'Month': pl.Int64,
                'Day': pl.Int64,
                'Hour': pl.Int64,
                'Minute': pl.Int64,
                'DNI': pl.Float64,
                'GHI': pl.Float64,
                'DHI': pl.Float64
            }
        )
    
    # Imprimir información de depuración
    print(f"\nInformación del archivo {archivo}:")
    print("Columnas:", df.columns)
    print("Primeras filas:")
    print(df.head())
    
    return df

def main():
    # Crear directorio de resultados si no existe
    os.makedirs('resultados', exist_ok=True)
    
    # Cargar datos de cada país
    datos_chile = cargar_datos('datos_Chile.csv')
    datos_china = cargar_datos('datos_China.csv')
    datos_sudafrica = cargar_datos('datos_Sudafrica.csv')
    
    # Calcular promedios y máximos
    promedios = {
        'Chile': {
            'GHI': float(datos_chile.select(pl.col('GHI').mean())[0,0]),
            'DHI': float(datos_chile.select(pl.col('DHI').mean())[0,0]),
            'DNI': float(datos_chile.select(pl.col('DNI').mean())[0,0])
        },
        'China': {
            'GHI': float(datos_china.select(pl.col('GHI').mean())[0,0]),
            'DHI': float(datos_china.select(pl.col('DHI').mean())[0,0]),
            'DNI': float(datos_china.select(pl.col('DNI').mean())[0,0])
        },
        'Sudáfrica': {
            'GHI': float(datos_sudafrica.select(pl.col('GHI').mean())[0,0]),
            'DHI': float(datos_sudafrica.select(pl.col('DHI').mean())[0,0]),
            'DNI': float(datos_sudafrica.select(pl.col('DNI').mean())[0,0])
        }
    }
    
    maximos = {
        'Chile': {
            'GHI': float(datos_chile.select(pl.col('GHI').max())[0,0]),
            'DHI': float(datos_chile.select(pl.col('DHI').max())[0,0]),
            'DNI': float(datos_chile.select(pl.col('DNI').max())[0,0])
        },
        'China': {
            'GHI': float(datos_china.select(pl.col('GHI').max())[0,0]),
            'DHI': float(datos_china.select(pl.col('DHI').max())[0,0]),
            'DNI': float(datos_china.select(pl.col('DNI').max())[0,0])
        },
        'Sudáfrica': {
            'GHI': float(datos_sudafrica.select(pl.col('GHI').max())[0,0]),
            'DHI': float(datos_sudafrica.select(pl.col('DHI').max())[0,0]),
            'DNI': float(datos_sudafrica.select(pl.col('DNI').max())[0,0])
        }
    }
    
    # Calcular irradiación diaria (kWh/m²/día)
    irradiacion_diaria = {
        'Chile': float(datos_chile.select(pl.col('GHI').sum())[0,0]) / (1000 * 365),
        'China': float(datos_china.select(pl.col('GHI').sum())[0,0]) / (1000 * 365),
        'Sudáfrica': float(datos_sudafrica.select(pl.col('GHI').sum())[0,0]) / (1000 * 365)
    }
    
    # Crear gráficos
    paises = list(promedios.keys())
    tipos = ['GHI', 'DHI', 'DNI']
    
    # Gráfico de promedios
    plt.figure(figsize=(12, 6))
    x = np.arange(len(paises))
    width = 0.25
    
    for i, tipo in enumerate(tipos):
        valores = [promedios[pais][tipo] for pais in paises]
        plt.bar(x + i*width, valores, width, label=tipo)
    
    plt.xlabel('País')
    plt.ylabel('Irradiancia Promedio (W/m²)')
    plt.title('Comparación de Irradiancias Promedio por País')
    plt.xticks(x + width, paises)
    plt.legend()
    plt.tight_layout()
    plt.savefig('resultados/comparacion_irradiancias.png')
    plt.close()
    
    # Gráfico de máximos
    plt.figure(figsize=(12, 6))
    for i, tipo in enumerate(tipos):
        valores = [maximos[pais][tipo] for pais in paises]
        plt.bar(x + i*width, valores, width, label=tipo)
    
    plt.xlabel('País')
    plt.ylabel('Irradiancia Máxima (W/m²)')
    plt.title('Comparación de Irradiancias Máximas por País')
    plt.xticks(x + width, paises)
    plt.legend()
    plt.tight_layout()
    plt.savefig('resultados/comparacion_maximos.png')
    plt.close()
    
    # Gráfico de irradiación diaria
    plt.figure(figsize=(10, 6))
    plt.bar(paises, list(irradiacion_diaria.values()))
    plt.xlabel('País')
    plt.ylabel('Irradiación Diaria Promedio (kWh/m²/día)')
    plt.title('Comparación de Irradiación Diaria Promedio por País')
    plt.tight_layout()
    plt.savefig('resultados/comparacion_irradiaciones.png')
    plt.close()
    
    # Imprimir resultados
    print("\nResultados de la comparación:")
    for pais in paises:
        print(f"\n{pais}:")
        print(f"GHI promedio: {promedios[pais]['GHI']:.2f} W/m² (máximo: {maximos[pais]['GHI']:.0f} W/m²)")
        print(f"DHI promedio: {promedios[pais]['DHI']:.2f} W/m² (máximo: {maximos[pais]['DHI']:.0f} W/m²)")
        print(f"DNI promedio: {promedios[pais]['DNI']:.2f} W/m² (máximo: {maximos[pais]['DNI']:.0f} W/m²)")
        print(f"Irradiación diaria promedio: {irradiacion_diaria[pais]:.3f} kWh/m²/día (GHI)")

if __name__ == "__main__":
    main() 

/tmp/ipykernel_746/152318661.py:9: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv(



Información del archivo datos_Chile.csv:
Columnas: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'GHI', 'DHI']
Primeras filas:
shape: (5, 8)
┌──────┬───────┬─────┬──────┬────────┬─────┬─────┬─────┐
│ Year ┆ Month ┆ Day ┆ Hour ┆ Minute ┆ DNI ┆ GHI ┆ DHI │
│ ---  ┆ ---   ┆ --- ┆ ---  ┆ ---    ┆ --- ┆ --- ┆ --- │
│ i64  ┆ i64   ┆ i64 ┆ i64  ┆ i64    ┆ f64 ┆ f64 ┆ f64 │
╞══════╪═══════╪═════╪══════╪════════╪═════╪═════╪═════╡
│ 2018 ┆ 1     ┆ 1   ┆ 0    ┆ 0      ┆ 0.0 ┆ 0.0 ┆ 0.0 │
│ 2018 ┆ 1     ┆ 1   ┆ 1    ┆ 0      ┆ 0.0 ┆ 0.0 ┆ 0.0 │
│ 2018 ┆ 1     ┆ 1   ┆ 2    ┆ 0      ┆ 0.0 ┆ 0.0 ┆ 0.0 │
│ 2018 ┆ 1     ┆ 1   ┆ 3    ┆ 0      ┆ 0.0 ┆ 0.0 ┆ 0.0 │
│ 2018 ┆ 1     ┆ 1   ┆ 4    ┆ 0      ┆ 0.0 ┆ 0.0 ┆ 0.0 │
└──────┴───────┴─────┴──────┴────────┴─────┴─────┴─────┘

Información del archivo datos_China.csv:
Columnas: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'GHI', 'DHI']
Primeras filas:
shape: (5, 8)
┌──────┬───────┬─────┬──────┬────────┬──────┬──────┬──────┐
│ Year ┆ Month ┆ Day 

---------------------------------------------------------------------------------------------

2) Simulación de Plantas PV y CSP


Tecnología PV

In [16]:
#Simulación de producción anual de energía. 500kW, 1MW, 5MW
import PySAM
